#### Importing all required Libraries

In [4]:
import os 

from dotenv import load_dotenv
from huggingface_hub import InferenceClient


load_dotenv()

True

#### Getting LLM Model: HF's Serverless API

In the HuggingFace ecosystem, there is a convenient feature called `Serverless API` that allows you to easily run inference on many models. There's no installation or deployment required.

In [6]:
client = InferenceClient("meta-llama/Llama-3.3-70B-Instruct")

In [12]:
output = client.text_generation(
    prompt="The capital of France is",
    max_new_tokens=100
)

print(output)

 a city that is steeped in history, art, fashion, and culture. From the iconic Eiffel Tower to the world-famous Louvre Museum, there are countless things to see and do in Paris. Here are some of the top attractions and experiences to add to your Parisian itinerary:
1. The Eiffel Tower: This iron lattice tower is one of the most recognizable landmarks in the world and offers breathtaking views of the city from its observation decks.
2. The Louvre Museum


As ween 

In [14]:
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
The capital of France is<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

output = client.text_generation(
    prompt,
    max_new_tokens=100
)

print(output)



The capital of France is Paris.


##### `Chat` Interface:
 
Using the `chat` method is a much more convenient and reliable way to apply chat templates:

In [15]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of France is"}
    ],
    stream=False,
    max_tokens=1024
)

print(output)

ChatCompletionOutput(choices=[ChatCompletionOutputComplete(finish_reason='stop', index=0, message=ChatCompletionOutputMessage(role='assistant', content='The capital of France is Paris.', tool_call_id=None, tool_calls=None), logprobs=None)], created=1749742141, id='', model='meta-llama/Llama-3.3-70B-Instruct', system_fingerprint='3.2.1-sha-4d28897', usage=ChatCompletionOutputUsage(completion_tokens=8, prompt_tokens=40, total_tokens=48), object='chat.completion')


In [16]:
print(output.choices[0].message.content)

The capital of France is Paris.


The chat method is the `RECOMMENDED` method to use in order to ensure a smooth transition between models

### Dummy Agent

In the previous section, we saw that the core of an agent library is to